___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___
# Question and Answer Chat Bots

----

------

In [1]:
import pickle
import numpy as np

In [2]:
with open("train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [3]:
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

## Exploring the Format of the Data

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(test_data)

1000

In [7]:
len(train_data)

10000

In [8]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [9]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [10]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [11]:
train_data[0][2]

'no'

-----

## Setting up Vocabulary of All Words

In [12]:
# Create a set that holds the vocab words
vocab = set()

In [13]:
all_data = test_data + train_data

In [14]:
for story, question , answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [15]:
vocab.add('no')
vocab.add('yes')

In [16]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [17]:
vocab_len = len(vocab) + 1 #we add an extra space to hold a 0 for Keras's pad_sequences

In [18]:
max_story_len = max([len(data[0]) for data in all_data])

In [19]:
max_story_len

156

In [20]:
max_question_len = max([len(data[1]) for data in all_data])

In [21]:
max_question_len

6

## Vectorizing the Data

In [22]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [23]:
# Reserve 0 for pad_sequences
vocab_size = len(vocab) + 1

In [24]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [25]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [26]:
tokenizer.word_index

{'journeyed': 1,
 'daniel': 2,
 'the': 3,
 'went': 4,
 'travelled': 5,
 'bathroom': 6,
 'down': 7,
 'mary': 8,
 'picked': 9,
 'dropped': 10,
 'there': 11,
 'to': 12,
 'apple': 13,
 'is': 14,
 'milk': 15,
 'hallway': 16,
 'back': 17,
 'sandra': 18,
 'john': 19,
 'left': 20,
 '?': 21,
 'kitchen': 22,
 'discarded': 23,
 'got': 24,
 '.': 25,
 'up': 26,
 'office': 27,
 'moved': 28,
 'football': 29,
 'garden': 30,
 'grabbed': 31,
 'in': 32,
 'no': 33,
 'put': 34,
 'took': 35,
 'yes': 36,
 'bedroom': 37}

In [27]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [28]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [29]:
len(train_story_text)

10000

In [30]:
len(train_story_seq)

10000

In [31]:
# word_index = tokenizer.word_index

### Functionalize Vectorization

In [32]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [33]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [34]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [35]:
inputs_test

array([[ 0,  0,  0, ...,  3, 37, 25],
       [ 0,  0,  0, ...,  3, 30, 25],
       [ 0,  0,  0, ...,  3, 30, 25],
       ...,
       [ 0,  0,  0, ...,  3, 13, 25],
       [ 0,  0,  0, ...,  3, 30, 25],
       [ 0,  0,  0, ..., 13, 11, 25]])

In [36]:
queries_test

array([[14, 19, 32,  3, 22, 21],
       [14, 19, 32,  3, 22, 21],
       [14, 19, 32,  3, 30, 21],
       ...,
       [14,  8, 32,  3, 37, 21],
       [14, 18, 32,  3, 30, 21],
       [14,  8, 32,  3, 30, 21]])

In [37]:
answers_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0.])

In [38]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
       503.,   0.,   0., 497.,   0.])

In [39]:
tokenizer.word_index['yes']

36

In [40]:
tokenizer.word_index['no']

33

## Creating the Model

In [41]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

In [42]:
# Placeholders
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

## Encoders

### Input Encoder m

In [43]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

### Input Encoder c

In [44]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

### Question Encoder

In [45]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

### Encode the Sequences

In [46]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

##### Use dot product to compute the match between first input vector seq and the query

In [47]:
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2,2))
match = Activation('softmax')(match)

#### Add this match matrix with the second input vector sequence

In [48]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

#### Concatenate

In [49]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

In [50]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [51]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)

In [52]:
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

In [53]:
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 156)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, 64)     2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
______________________________________________________________________________________________

In [55]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/120
313/313 [==============================] - 5s 10ms/step - loss: 0.8809 - accuracy: 0.4966 - val_loss: 0.7053 - val_accuracy: 0.4970
Epoch 2/120
313/313 [==============================] - 3s 10ms/step - loss: 0.7007 - accuracy: 0.5056 - val_loss: 0.6994 - val_accuracy: 0.4970
Epoch 3/120
313/313 [==============================] - 4s 14ms/step - loss: 0.6955 - accuracy: 0.5004 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 4/120
313/313 [==============================] - 4s 14ms/step - loss: 0.6950 - accuracy: 0.4934 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 5/120
313/313 [==============================] - 5s 15ms/step - loss: 0.6945 - accuracy: 0.5006 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 6/120
313/313 [==============================] - 4s 14ms/step - loss: 0.6949 - accuracy: 0.4971 - val_loss: 0.6956 - val_accuracy: 0.4970
Epoch 7/120
313/313 [==============================] - 4s 14ms/step - loss: 0.6946 - accuracy: 0.4987 - val_loss: 0.6938 - val_accuracy:

313/313 [==============================] - 4s 14ms/step - loss: 0.2936 - accuracy: 0.8721 - val_loss: 0.3765 - val_accuracy: 0.8360
Epoch 58/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2944 - accuracy: 0.8724 - val_loss: 0.3761 - val_accuracy: 0.8420
Epoch 59/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2909 - accuracy: 0.8744 - val_loss: 0.3958 - val_accuracy: 0.8450
Epoch 60/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2889 - accuracy: 0.8753 - val_loss: 0.3861 - val_accuracy: 0.8330
Epoch 61/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2920 - accuracy: 0.8741 - val_loss: 0.3571 - val_accuracy: 0.8380
Epoch 62/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2805 - accuracy: 0.8775 - val_loss: 0.3654 - val_accuracy: 0.8340
Epoch 63/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2835 - accuracy: 0.8752 - val_loss: 0.3667 - val_accuracy: 0.836

313/313 [==============================] - 4s 14ms/step - loss: 0.2295 - accuracy: 0.9048 - val_loss: 0.4992 - val_accuracy: 0.8280
Epoch 114/120
313/313 [==============================] - 5s 15ms/step - loss: 0.2292 - accuracy: 0.9019 - val_loss: 0.4564 - val_accuracy: 0.8350
Epoch 115/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2281 - accuracy: 0.9035 - val_loss: 0.4762 - val_accuracy: 0.8300
Epoch 116/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2278 - accuracy: 0.9027 - val_loss: 0.5174 - val_accuracy: 0.8230
Epoch 117/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2189 - accuracy: 0.9048 - val_loss: 0.5160 - val_accuracy: 0.8330
Epoch 118/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2267 - accuracy: 0.9065 - val_loss: 0.5351 - val_accuracy: 0.8340
Epoch 119/120
313/313 [==============================] - 4s 14ms/step - loss: 0.2157 - accuracy: 0.9076 - val_loss: 0.5112 - val_accuracy:

### Saving the Model

In [57]:
filename = 'chatbot_120_epochs.h5'
model.save(filename)

## Evaluating the Model


### Evaluating on Given Test Set

In [58]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

In [59]:
test_data[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [60]:
story =' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [61]:
query = ' '.join(word for word in test_data[0][1])
print(query)

Is John in the kitchen ?


In [62]:
print("True Test Answer from Data is:",test_data[0][2])

True Test Answer from Data is: no


In [63]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.9957195


## Writing Your Own Stories and Questions



In [64]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [65]:
# Note the whitespace of the periods
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [66]:
my_question = "Is the football in the garden ?"

In [67]:
my_question.split()

['Is', 'the', 'football', 'in', 'the', 'garden', '?']

In [68]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [69]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [70]:
pred_results = model.predict(([ my_story, my_ques]))

In [71]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.99955815
